In [2]:
#호주 문건 10개만
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('window-size=1920x1080')
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(options=options)

In [3]:
import time

driver.get('https://www.abf.gov.au/help-and-support/notices/australian-customs-notices')
time.sleep(2)

In [4]:
soup = BeautifulSoup(driver.page_source, 'lxml')

In [5]:
temp = soup.find('ul', attrs={'class' : 'pagination'}).find_all("li")
# len(temp)

In [6]:
from selenium.webdriver.common.by import By
import re

In [7]:
dates = []
titles = []
links = []
last_page = 10

for i in range(2,last_page):
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    date_rows = soup.select('div.col-sm-3 span')
    title_rows = soup.find('tbody').find_all('tr', attrs = {'class' : re.compile('^accordion-header')})
    link_rows = soup.find('tbody').find_all('tr', attrs = {'class' : re.compile('^accordion-content')})
    
    for row in date_rows:
        date = row.text
        dates.append(date)

    for row in title_rows:
        tds = row.find_all('td')
        title = tds[1].get_text().replace('Press Enter to show more details.', '')  
        titles.append(title)

    for row in link_rows:
        link = 'https://www.abf.gov.au' + row.a['href']
        links.append(link)
    
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(2)
    
    elem = driver.find_element(By.LINK_TEXT,str(i))
    elem.click()
    time.sleep(1)

In [9]:
soup = BeautifulSoup(driver.page_source, 'lxml')
    
date_rows = soup.select('div.col-sm-3 span')
title_rows = soup.find('tbody').find_all('tr', attrs = {'class' : re.compile('^accordion-header')})
link_rows = soup.find('tbody').find_all('tr', attrs = {'class' : re.compile('^accordion-content')})
    
for row in date_rows:
    date = row.text
    dates.append(date)

for row in title_rows:
    tds = row.find_all('td')
    title = tds[1].get_text().replace('Press Enter to show more details.', '')  
    titles.append(title)

for row in link_rows:
    link = 'https://www.abf.gov.au' + row.a['href']
    links.append(link)

In [10]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter
from pdfminer.pdfpage import PDFPage
import io
import urllib.request
import requests


def pdf_to_text(pdf_file):
    text_memory_file = io.StringIO()

    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, text_memory_file, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    # get first 3 pages of the pdf file
    for page in PDFPage.get_pages(pdf_file, pagenos=(0,0)):
        interpreter.process_page(page)
    text = text_memory_file.getvalue()
    text_memory_file.close()
    return text



In [13]:
texts = []
start = 0
finish = 10



for i in range(start, finish): # 100개씩 끊어서 돌린 뒤 texts에 추가, 총 1683개의 링크, 17번 돌리기
    url = links[i]
    response = requests.get(url)
    my_raw_data = response.content
    
    print(i+1, "번 pdf 진행 중")
    # online pdf to text by requests
    response = requests.get(url)
    pdf_memory_file = io.BytesIO()
    pdf_memory_file.write(response.content)
    texts.append(pdf_to_text(pdf_memory_file))


1 번 pdf 진행 중
2 번 pdf 진행 중
3 번 pdf 진행 중
4 번 pdf 진행 중
5 번 pdf 진행 중
6 번 pdf 진행 중
7 번 pdf 진행 중
8 번 pdf 진행 중
9 번 pdf 진행 중
10 번 pdf 진행 중


In [14]:
import pandas as pd

df = pd.DataFrame()
df['date'] = pd.to_datetime(dates)[:10]
df['link'] = links[:10]
df['title'] = titles[:10]
df['text'] = texts

df

,date,link,title,text
0,2022-07-28,https://www.abf.gov.au/help-and-support-subsit...,Indexation of customs duty rates on excise-equ...,Australian Customs Notice \nNo. 2022/35 \nInd...
1,2022-07-22,https://www.abf.gov.au/help-and-support-subsit...,Removal of customs duty on certain electric ve...,Australian Customs Notice \nNo. 2022/34 \nRem...
2,2022-07-21,https://www.abf.gov.au/help-and-support-subsit...,Application for Customs Broker Licences,Australian Customs Notice No. 2022/33 \n\nAppl...
3,2022-07-12,https://www.abf.gov.au/help-and-support-subsit...,Temporary duty reduction for goods from Ukraine,Australian Customs Notice \nNo. 2022/32 \nTem...
4,2022-07-11,https://www.abf.gov.au/help-and-support-subsit...,Application for Customs Broker Licences,Australian Customs Notice No. 2022/31 \n\nAppl...
5,2022-07-01,https://www.abf.gov.au/help-and-support-subsit...,Cargo reporting requirements for international...,Australian Customs Notice (ACN) \n2022/30 \nC...
6,2022-07-01,https://www.abf.gov.au/help-and-support-subsit...,New timeframes for the disposal of unentered a...,Australian Customs Notice (ACN) \n2022/29 \n\n...
7,2022-06-29,https://www.abf.gov.au/help-and-support-subsit...,Tariff Concession – Item 57 – Revised,Australian Customs Notice \nNo. 2022/28 \nTar...
8,2022-06-28,https://www.abf.gov.au/help-and-support-subsit...,Commencement of phase 2 of the Recycling and W...,Australian Customs Notice \nNo. 2022/27 \nCom...
9,2022-06-27,https://www.abf.gov.au/help-and-support-subsit...,Application for Customs Broker Licences,Australian Customs Notice No. 2022/26 \n\nAppl...


In [15]:
df.to_csv('호주_세관_최근10.csv', mode='w', encoding='utf-8-sig', index = False)